<a href="https://colab.research.google.com/github/Lau-Jr/data-engineering-dump/blob/main/dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
sc = SparkContext()
spark = SparkSession(sc)


In [ ]:

data = [
        ('laurent',25,'asha'),
        ('diana',21,'asha'),
        ('yusra',16,'asha'),
        ('france',15,'ngara'),
        ('zion',12,'upendo'),
        ('pu',10,'upendo'),
        ('prince',7,'upendo'),
        ('gato',3,'upendo'),
        ('edo',1,'upendo'),
        ]
names = ['name','age','mother']
dataRDD = sc.parallelize(data)

In [ ]:
watotoDF = dataRDD.toDF()
new_watotoDF = spark.createDataFrame(data,schema = names)

In [ ]:
house_df = spark.read.csv('/content/sample_data/california_housing_train.csv',header = True,inferSchema =True)
house_df.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 2 rows



In [ ]:
ansombe_js = spark.read.json('/content/sample_data/anscombe.json')
ansombe_js.show(2)

+------+----+----+---------------+
|Series|   X|   Y|_corrupt_record|
+------+----+----+---------------+
|  null|null|null|              [|
|     I|10.0|8.04|           null|
+------+----+----+---------------+
only showing top 2 rows



In [ ]:
house_df.select(['total_rooms','total_bedrooms']).show(2)

+-----------+--------------+
|total_rooms|total_bedrooms|
+-----------+--------------+
|     5612.0|        1283.0|
|     7650.0|        1901.0|
+-----------+--------------+
only showing top 2 rows



In [ ]:
house_df.filter(house_df.total_rooms > 5000).show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -116.06|   34.15|              15.0|    10377.0|        2331.0|    4507.0|    1807.0|       2.2466|           66800.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 3 rows



In [ ]:
house_df.groupBy('latitude').count().orderBy('count',ascending = False).count()

840

In [ ]:
house_df.groupBy('latitude').count().dropDuplicates().orderBy('count',ascending = False).count()

840

In [ ]:
watotoDF = watotoDF.withColumnRenamed("_3",'mother')

In [ ]:
watotoDF.columns

['_1', '_2', 'mother']

In [ ]:
watotoDF.createOrReplaceTempView('kids')

In [ ]:
spark.sql('select * from kids where mother = "asha" ').show()

+-------+---+------+
|     _1| _2|mother|
+-------+---+------+
|laurent| 25|  asha|
|  diana| 21|  asha|
|  yusra| 16|  asha|
+-------+---+------+



In [ ]:
watotoDF.collect()[0].asDict()['mother']

'asha'

In [ ]:
watotoDF.groupBy('Mother').count().orderBy('count',ascending = False).show()

+------+-----+
|Mother|count|
+------+-----+
|upendo|    5|
|  asha|    3|
| ngara|    1|
+------+-----+



###RENAMING COLUMNS

In [ ]:
# Average duration of Delta flights
flights.filter(flights.carrier == "DL").filter(flights.origin == "SEA").groupBy().avg("air_time").show()

# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show()

### from pyspark.sql.functions import stddev

In [ ]:
# Import pyspark.sql.functions as F
from pyspark.sql.functions import stddev as std

# Group by month and dest
by_month_dest = flights.groupBy('month','dest')

# Average departure delay by month and destination
by_month_dest.avg('dep_delay').show()

# Standard deviation of departure delay
by_month_dest.agg(std('dep_delay')).show()

##joins in pyspark

In [ ]:
# Examine the data
print(airports.show())

# Rename the faa column
airports = airports.withColumnRenamed('faa','dest')

# Join the DataFrames
flights_with_airports = flights.join(airports,on='dest',how='leftouter')

# Examine the new DataFrame
print(flights_with_airports.show())

##type conversion

In [ ]:
# Cast the columns to integers
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("month", model_data.month.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

##selecting

In [ ]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")